In [ ]:
import numpy as np
import torch 
import torch.nn as nn
import pandas as pd

In [ ]:
class VolatilityModel(nn.Module):
    def __init__(self, num_stocks, feature_dim):
        super().__init__()
        self.num_stocks = num_stocks
        self.feature_dim = feature_dim
        
        self.stock_embedding = nn.Embedding(self.num_stocks, 8)
        self.gru = nn.GRU(feature_dim=self.feature_dim, hidden_size=64, num_layers=1, batch_first=True)
        
        self.regressor = nn.Sequential(
            nn.Linear(64 + 8, 128),
            nn.GELU(),
            nn.Linear(128, 1)
        )

In [ ]:
# (seq_len, features_dim) 

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims: list, output_dim: int, dropout=0.10):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_layers = nn.ModuleList()
        
        fan_in = self.input_dim
        for hidden_dim in hidden_dims:
            self.hidden_layers.append(nn.Linear(fan_in, hidden_dim))
            fan_in = hidden_dim
        self.hidden_layers.append(nn.Linear(fan_in, self.output_dim))
        
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(p=dropout)
        
    @staticmethod
    def criterion(pred, y):
        loss = torch.mean((1 - (pred / y))**2)
        return loss
        
    def forward(self, x):
        for layer in self.hidden_layers[:-1]:
            x = layer(x)
            x = self.gelu(x)
            x = self.dropout(x)
            
        x = self.hidden_layers[-1](x)
        return x

In [ ]:
df = pd.read_csv('../baseline2.csv', index_col=False).iloc[:, 1:]
df.shape

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['stock_id', 'time_id', 'target'])
y = df['target']

X = X.to_numpy()
y = y.to_numpy()

X = torch.tensor(X).to(torch.float)
y = torch.tensor(y).to(torch.float) * 10000

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, pin_memory=True)

In [ ]:
INPUT_DIM = X_train.shape[1]
OUTPUT_DIM = 1
HIDDEN_DIMS = [256, 256, 256]
DROPOUT_RATE = 0.1

In [ ]:
from torch.optim import Adam

In [ ]:
lr = 5e-4
criterion = SimpleMLP.criterion
model = SimpleMLP(INPUT_DIM, HIDDEN_DIMS, OUTPUT_DIM, DROPOUT_RATE)
optimizer = Adam(model.parameters(), lr=lr)
num_epochs = 100
device = 'cuda'

model = model.to(device)
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for i, (batch_x, batch_y) in enumerate(dataloader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        preds = model(batch_x)
        loss = criterion(preds, batch_y)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}.')